# Workshop 1: Analyzing connectomics datasets: V1dd

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
   
<font size=4> The function of the nervous system arises out of a combination of the properties of individual neurons and the properties of how they are connected into a larger network. The central goal of <b>connectomics</b> is to produce complete maps of the connectivity of the nervous system with synaptic resolution and analyze them to better understand the organization, development, and function of the nervous system. </font>

<font size=4><b>Electron Microscopy (EM) data enables morphological reconstruction of neurons and resolution of their synaptic connectivity </b>. The V1DD dataset is one of the largest volume EM datasets currently available, and spans all layers of mouse visual cortex. We will be using this dataset to query the connectivity between neurons in the visual cortex. </font>

<b> The V1DD dataset is newly public. </b> Previous years of this course have used the [MICrONS Dataset](https://www.microns-explorer.org/), which is also a cubic millimeter volume of mouse visual cortex. Both datasets are available for the use of this course, but we will cover V1DD in Week 1.

<h3> Workshop aims</h3>
<ul>
    <li> Understand what are the major classes of cell types in cortex</li>
    <li> Understand the basics of how synaptic connectivity is measured in EM connectomics</li>
    <li> Look at the relationship between the structure of networks and cell type connectivity</li>
    <li> Look at the relationship between functional tuning properties of neurons and how they are connected in the network </li>
</ul>
    

<h4> Workshop 1 will cover: </h4>
<ol> 
    <li>Reconstructions of individual neurons</li>
    <li>Connectivity of individual neurons</li>
    <li>Connectivity between cell types</li>
    <li>Connectivity by target type (spine, soma, shaft)</li>
</ol>

<h4> Workshop 2 will cover: </h4>
<ol> 
    <li>Connection probability as a function of distance</li>
    <li>The relationship between structure and function </li>
    <li>Designing null models to test for meaningful connectivity </li>
</ol>

<em> Note on data access: </em> To make this workshop easier, we already queried the most of the data needed for this exercise from the database. We have made it available as versioned files that can be read with pandas. The entire dataset is hosted using the <a href=https://caveconnectome.github.io/sections/cave_overview.html> Connectome Annotation Versioning Engine (CAVE) </a>. A separate notebook shows how to use CAVE to generate the files used in this notebook: `code/solutions/DataProducts.ipynb`
    
    
</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

This cell sets up a variable called `data_root` that generalizes the data directory across systems, whether you are using CodeOcean or the prepared data drives

</div>

In [ ]:
import sys
from os.path import join as pjoin
import platform

# Add the directory for the data and utilities
mat_version = 1196

platstring = platform.platform()
system = platform.system()
if system == "Darwin":
    # macOS
    data_root = "/Volumes/Brain2025/"
elif system == "Windows":
    # Windows (replace with the drive letter of USB drive)
    data_root = "E:/"
elif "amzn" in platstring:
    # then on CodeOcean
    data_root = "/data/"
else:
    # then your own linux platform
    # EDIT location where you mounted hard drive
    data_root = "/media/$USERNAME/Brain2025/"

# Set the directory to load prepared data and utility code
data_dir = pjoin(data_root, f"v1dd_{mat_version}")
utils_dir = pjoin("utils")

# Add utilities to path
sys.path.append(utils_dir)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<h2> Import packages </h2>
   
<b> The CAVEclient</b> is a python library that facilitates communication with a CAVE system. For convenience, we also use the package <b>skeleton_plot</b> which handles rendering the precomputed skeletons. 
   
</div>

In [ ]:
# Import packages
from caveclient import CAVEclient
import skeleton_plot as skelplot

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from scipy.sparse import csr_array
from typing import Union, Optional
from utils import check_index, adjacencyplot

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
   
<b> CAVE account setup</b>

<p>In order to manage server traffic, every user needs to create a CAVE account and download a user token to access CAVE's services programmatically. The CAVE infrastructure can be read about in <a href='https://doi.org/10.1038/s41592-024-02426-z'>more detail in the MICrONS Nature Package</a>. 

<b> A Google account (or Google-enabled account) is required to create a CAVE account.</b>

Run the cell below and follow the set up instructions; enter the token first, then enter 'y' when prompted

</div>

In [ ]:
# Token Generation and Setup
CAVEclient.setup_token("https://global.em.brain.allentech.org")

# Initialize CAVEclient
client = CAVEclient(datastack_name="v1dd_public")

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<p><b>Tip for CodeOcean users</b>: you can add the the CAVEclient token to your capsule's secrets, so you do not have to enter it every time you run the notebook. If you chose to use EM data for your project in Week 2 we can show you how. 

</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

    
## Interactive session in neuroglancer

Instructions in the <a href=https://allenswdb.github.io/anatomy/microns-em/em-neuroglancer.html> DataBook: Neuroglancer </a>

<font size=4> <a href=https://spelunker.cave-explorer.org/#!middleauth+https://global.daf-apis.com/nglstate/api/v1/4876435155058688> V1DD Neuroglancer Explorer </a> </font>

You will be prompted for a google login. Use the same account you used to authenticate your CAVE token
    
</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

    
## Proofreading and data quality

Understanding this variablity in data quality is critical when interpretting electron microscopy reconstructions.

Automated segmentation of neuronal processes in dense EM imaging is challenging at the size of entire neurons, which can have millimeters of axons and dendrites. The automated segmentation algorithms used in the EM data for this project are not perfect, and so proofreading is necessary to obtain accurate reconstructions of a cell and confidence in the connectivity

In general, the **dendrites** of single-soma detected neurons are considered trustworthy, even without proofreading. However the **axons** require manual effort to both clean and extend. 

For details on the proofreading status see: 

*  `code/solutions/DataProducts.ipynb` for how the data we use today was generated
* Proofreading page in the <a href=https://allenswdb.github.io/anatomy/microns-em/proofreading.html> SWDB databook</a>

<b> For this workshop, we treat all cells with at least `axon_partially_extended` as equally trustworth.</b> This may not be a safe assumption for all analysis, and we are happy to provide more guidance depending on the research question.

<font size=3>Load the prepared variables: <code>dendrite_proof_root_ids</code> and <code>axon_proof_root_ids</code> </font>
    
</div>

In [ ]:
# Loads cells with axon and dendrite proofreading
dendrite_proof_root_ids = np.load(
    pjoin(data_dir, f"proofread_dendrite_list_{mat_version}.npy")
)
axon_proof_root_ids = np.load(pjoin(data_dir, f"proofread_axon_list_{mat_version}.npy"))

print(
    f"There are {len(dendrite_proof_root_ids)} cells with acceptable dendrites, and {len(axon_proof_root_ids)} cells with axon proofreading"
)
print(
    f"The number of cells with usable axons and dendrites is: {(np.isin(axon_proof_root_ids, dendrite_proof_root_ids).sum())}"
)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
    
## Neuron morphology: representing neurons as skeletons

Often in thinking about neurons, you want to measure things along a linear dimension of a neuron.

However, the segmentation and meshes are a complex 3D shape that makes this non-trivial. There are methods for reducing the shape of a segmented neuron down to a linear tree like structure usually referred to as a **skeleton**. We have precalculated skeletons for a large number of cells in the dataset, and make the skeleton generation available on our server, on demand.

*Note*: The meshes you see in Neuroglancer are available to download through the python client `cloud-volume`, and can be loaded for analysis and visualization in other tools. This is useful for some types of analysis (and for making beautiful renders of neurons), but we will not cover that in this course.

<font size=4> First, define a convenience function for plotting skeletons. </font>
    
</div>

In [ ]:
# Color map for the different comparments. You may adjust these to your asthetic.
skel_color_map = {3: "firebrick", 4: "salmon", 2: "steelblue", 1: "olive"}


def plot_skeleton(sk_d: dict, plot_soma=True, ax=None):
    """Plots a skeleton.

    Args:
        sk: skeleton dict as returned from CAVEclient

    Returns:
        ax: plot axes
    """
    if ax is None:
        fig, ax = plt.subplots(figsize=(7, 10), dpi=150)

    skelplot.plot_tools.plot_verts(
        np.array(sk_d["vertices"]),
        np.array(sk_d["edges"]),
        radius=np.array(sk_d["radius"]) / 1000 * 1,
        ax=ax,
        skel_colors=sk_d["compartment"],
        title="",
        skel_alpha=1.0,
        line_width=5,
        x="x",  # You can alter the projection direction by changing this to `y` or `z`
        y="y",  # You can alter the projection direction by changing this to `x` or `z`
        plot_soma=plot_soma,
        soma_node=sk_d["root"],
        color="darkslategray",
        soma_size=120,
        invert_y=True,
        skel_color_map=skel_color_map,
        x_min_max=None,
        y_min_max=None,
        capstyle="round",
        joinstyle="round",
    )

    ax.spines["right"].set_visible(False)
    ax.spines["left"].set_visible(False)
    ax.spines["top"].set_visible(False)
    ax.spines["bottom"].set_visible(False)
    ax.axis("off")

    return ax

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<font size=4> Select one of the axon-proofread cells to plot as a skeleton. </font>

</div>

In [ ]:
# select from the cells which are fully extended
root_id = axon_proof_root_ids[1]  # iterate this number to choose a different cell

# Query the skeleton from CAVEclient
sk_d = client.skeleton.get_skeleton(root_id, output_format="dict")

In [ ]:
# Plot skeleton in 2D
ax = plot_skeleton(sk_d)
plt.show()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<h4> Morphological compartment names </h4>

<p> The compartment types defined at each vertex adhere to standardized conventions for neuronal morphologies saved as swc files (for more information, see here: www.neuromorpho.org). </p>
    
These conventions are as follows:
<ul> 
    <li> 0 - undefined </li>
    <li> 1 - soma (default color 'olive') </li>
    <li> 2 - axon (default color 'steelblue') </li>
    <li> 3 - (basal) dendrite (default color 'firebrick' red) </li>
    <li> 4 - apical dendrite </li>
    <li> 5+ - custom </li>
</ul>

In most of these neurons, distinctions were not made between basal or apical dendrites - therefore dendrites should almost exclusively map to compartment type "3".

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Task 1:</b> Plot several different skeletons by changing the <code> root_id </code> entry. You can either iterate through the <code>axon_proof_root_ids</code> list, or paste the segment id from neuroglancer.

<b>Discussion:</b> What information do you lose going from 3D neuron meshes (neuroglancer) to 2D skeleton projections? What becomes more clear?
    
</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

    
## Synapse information

While synapses are stored as any other CAVE table in the database, in this case `synapses_v1dd`, this table is much larger than any other table at almost <b>640 million rows</b>, and it works best when queried in a directed way.

For today's workshop we have collected all of the input and output synapses for the proofread cells When accessing CAVE, the `synapse_query()` function allows you to query the synapse table in a more convenient way than most other tables. In particular, the `pre_ids` and `post_ids` let you specify which root id (or collection of root ids) you want to query, with pre_ids indicating the collection of presynaptic neurons and post_ids the collection of postsynaptic neurons.

Note that synapse queries always return the list of every synapse between the neurons in the query, even if there are multiple synapses between the same pair of neurons.

A common pattern to generate a list of connections between unique pairs of neurons is to group by the root ids of the presynaptic and postsynaptic neurons and then count the number of synapses between them. 

<font size=4> Load the table of connections between proofread cells (about <b> 8 million </b> entries) </font>
    
</div>

In [ ]:
syn_df = pd.read_feather(
    f"{data_dir}/syn_df_all_to_proofread_to_all_{mat_version}.feather"
)

print(syn_df.shape)

syn_df.head(3)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

    

<ul>This table has a number of columns, we will highlight the most important
    <li><b> id:</b> a unique ID for each synapse </li>
    <li><b> pre_pt_root_id:</b> the segmentation ID of the pre-synaptic compartment</li>
    <li><b> post_pt_root_id:</b> the segmentation ID of the post-synaptic compartment</li>
    <li><b> size:</b> a measure of the synapse size (the number of 4,4,40 nm voxels in the synapse mask) best available metric of synaptic weight</li>
    <li><b> ctr_pt_position_{x,y,z}:</b> the location of the synapse in the cleft, stored here in nanometers</li>
    <li><b> pre_pt_position_{x,y,z}:</b> a point just in the presynaptic compartment of synapse (used to lookup pre_root_id), stored here in nanometers</li>
    <li><b> post_pt_position_{x,y,z}:</b> a point just in the postsynaptic compartment of synapse (used to lookup post_pt_root_id), stored here in nanomaters</li>
</ul>

</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

    
## Combining neuron morphology & synapses

<p> Next we will visualize the synapses on the skeleton of your cell of interest. </p>

<p> Here we define a convenience function for robustly filtering the synapse table by pre- or post-synaptic partners. You will use this through the rest of the exercises. </p> 
    
</div>

In [ ]:
def filter_synapse_table(
    synapse_table: pd.DataFrame, pre_root_ids=None, post_root_ids=None
):
    """Filter synapse table by pre and post root ids.

    Args:
        synapse_table: synapse table with pre_pt_root_ids and post_pt_root_ids as pd.DataFrame
        pre_root_ids: np.ndarray, list or pd.Series if root_ids to filter on the presynaptic side
        post_root_ids: np.ndarray, list or pd.Series if root_ids to filter on the postsynaptic side

    Returns:
        synapse_table: filtered synapse table
    """

    if pre_root_ids is not None:
        assert isinstance(pre_root_ids, (np.ndarray, list, pd.core.series.Series)), (
            f"IDs have to be of type np.ndarray, list or pd.Series; got {type(pre_root_ids)}"
        )
        pre_mask = np.isin(synapse_table["pre_pt_root_id"], pre_root_ids)
    else:
        pre_mask = np.ones(len(synapse_table), dtype=bool)

    if post_root_ids is not None:
        assert isinstance(post_root_ids, (np.ndarray, list, pd.core.series.Series)), (
            f"IDs have to be of type np.ndarray, list or pd.Series; got {type(pre_root_ids)}"
        )
        post_mask = np.isin(synapse_table["post_pt_root_id"], post_root_ids)
    else:
        post_mask = np.ones(len(synapse_table), dtype=bool)

    return synapse_table[pre_mask & post_mask]


<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

    
<h3> Collect the output synapses for your cell of interest. </h3>

The nomenclature here is that you want synapses where your cell is the **presynaptic** partner

</div>

In [ ]:
# select from the cells which are fully extended
root_id = axon_proof_root_ids[1]  # iterate this number to choose a different cell

# Query the skeleton from CAVEclient
sk_d = client.skeleton.get_skeleton(root_id, output_format="dict")

pre_syns = filter_synapse_table(syn_df, pre_root_ids=[root_id])

pre_syns.head(5)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

    
Add the synapse positions to yours skeleton. We use:

<ul>
    <li><b> ctr_pt_position_{x,y,z}:</b> the location of the synapse in the cleft, stored here in nanometers</li>
</ul>

</div>

In [ ]:
# Plot your skeleton and overlay synapses
ax = plot_skeleton(sk_d)

sns.scatterplot(
    data=pre_syns,
    x="ctr_pt_position_x",
    y="ctr_pt_position_y",
    s=5,
    color="b",
    ax=ax,
    edgecolor=None,
    zorder=100,
)

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Task 2:</b> Now use the filter function above to create a dataframe of all postsynaptic synapses and create a version of the plot above that includes these input synapses. 
    
</div>

In [ ]:
# Select the synapses where your root_id is postsynaptic ('downstream'); the inputs to your cell


# Plot skeleton with input and output synapses

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Task 3:</b> Repeat the plotting code up to this point with a different neuron, changing the <code> root_id </code> variable above. 

<p> Try several, until you see something you haven't seen before. </p>

<b> Discussion </b>: what do you notice about the distribution of synapses relative to the soma of different cells? Above/below? Narrow or broad? What about the density of synapses on parts of the dendrites or axons?

How would you quantify these features?
    
</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

## Synapse matrix

The synapses between neurons create a network of synaptic connections. One way of visualizing this connectivity is in matrix form. 

For now, we will limit ourselves to the synapses between the proofread neurons.

</d>

In [ ]:
# get the ids of all proofread cells with both axon and dendrite proofreading
proof_root_ids = axon_proof_root_ids[
    np.isin(axon_proof_root_ids, dendrite_proof_root_ids)
]

# filter for synapses between the proofread cells (pre and post)
proof_proof_syn_table = filter_synapse_table(
    syn_df, pre_root_ids=proof_root_ids, post_root_ids=proof_root_ids
)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

Next we will make an **adjacency plot** from the tabular synapse data, turning it into a matrix the describes the connection strength between all possible partners. For convenience we have provided a function `make_adjacency` to perform the ordered groupbys and aggregations. 

</d>

In [ ]:
from utils import check_index


def make_adjacency(
    synapses,
    source_cell_index: Union[pd.Index, pd.DataFrame, pd.Series, np.ndarray, list],
    target_cell_index: Optional[
        Union[pd.Index, pd.DataFrame, pd.Series, np.ndarray, list]
    ] = None,
    aggfunc="binary",
    return_as="dense",
) -> Union[csr_array, np.ndarray]:
    source_cell_index = check_index(source_cell_index)
    if target_cell_index is None:
        target_cell_index = source_cell_index
    else:
        target_cell_index = check_index(target_cell_index)
    synapses = synapses.query(
        "pre_pt_root_id in @source_cell_index and post_pt_root_id in @target_cell_index"
    )
    groupby = synapses.groupby(["pre_pt_root_id", "post_pt_root_id"])
    if aggfunc == "count":
        edges = groupby.size().rename("weight").reset_index()
    elif aggfunc == "binary":
        edges = groupby.size().transform(lambda x: x > 0).rename("weight").reset_index()
    else:
        edges = groupby["size"].agg(aggfunc).rename("weight").reset_index()
    # make sure that the adjacency matrix is sorted the same as the input cell index
    edges["source_index"] = source_cell_index.get_indexer(edges["pre_pt_root_id"])
    edges["target_index"] = target_cell_index.get_indexer(edges["post_pt_root_id"])
    adjacency = csr_array(
        (edges["weight"], (edges["source_index"], edges["target_index"])),
        shape=(len(source_cell_index), len(target_cell_index)),
        dtype=edges["weight"].dtype,
    )
    # NOTE: for many applications working with sparse matrices is more efficient
    # but for ease of use and visualization in this workshop we return a dense matrix
    if return_as == "dense":
        adjacency = adjacency.todense()
    elif return_as == "sparse":
        pass  # already in sparse format
    else:
        raise ValueError(f"Unknown return_as type: {return_as}")
    return adjacency


<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

Each synapse has a `size` value assigned to it. How to aggregate the sizes from multiple synapes between two neurons depends on the research question. Synapse sizes vary by a lot and are related to the physiological strength of a synapse ([Holler et al., 2021](https://www.nature.com/articles/s41586-020-03134-2)). 

<font size=4> First, we will ignore the synapse size and strength and only look at <b>binary (connected yes/no) connectivity</b>. There are other measures of connectivity you can explore below that changes how the network appears. </font3>

</d>

In [ ]:
syn_mat = make_adjacency(proof_proof_syn_table, proof_root_ids, aggfunc="binary")
syn_mat

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

It looks like many of the values in the adjacency matrix array are `False`. In other words, most of the neurons in the proofread dataset are not connected to each other. How can we look at the places where there are connections? We can use the `np.nonzero()` function to get the indices of the non-zero (`True`) values in the array.  

</div>

In [ ]:
# show the first 10 nonzero values in the array
row_indices, column_indices = np.nonzero(syn_mat)
print("Row indices:", row_indices[:10])
print("Column indices:", column_indices[:10])
print("Matrix values:", syn_mat[row_indices[:10], column_indices[:10]])

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

<b> Discussion </b>: what does it mean that all of the first 10 values we looked at share the same row index?

</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

Now, let's move on to plotting the entire matrix.

</div>

In [ ]:
sns.set_context("paper", font_scale=1.0)


fig, ax = plt.subplots(figsize=(6, 6), dpi=200)

sns.heatmap(
    syn_mat,
    cbar=False,
    cmap="Greys",
    xticklabels=False,
    yticklabels=False,
    square=True,
    ax=ax,
)
ax.spines[["left", "right", "top", "bottom"]].set_visible(True)
_ = ax.set(
    xlabel="Postsynaptic cell",
    ylabel="Presynaptic cell",
    title="Adjacency matrix between proofread cells",
)

In [ ]:
n_edges = len(row_indices)
n_possible_edges = syn_mat.shape[0] * syn_mat.shape[1]
print(f"Number of edges: {n_edges}")
print(f"Number of possible edges: {n_possible_edges}")
print(f"Fraction of possible edges: {n_edges / n_possible_edges:.4f}")

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Task 3:</b> The matrix above only contains connections between <b>axon-proofread neurons</b>. Because many more dendrites of cells in the V1DD dataset are well-reconstructed <b>we can consider more connections than just the ones between proofread neurons.</b> Therefore, the connectivity matrix of analyzable connections forms a rectangular matrix. 
    
What is the size of the matrix between the proofread neurons and all neurons with a cell body in the dataset?
    
</div>

In [ ]:
# Select synapses from proofread cells onto all cells

# make adjanceny matrix from proofread cells onto all cells

# print shape of the matrix

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<h4> Consider: different measures of synaptic strength </h4>

When creating a connectivity matrix, how you measure synaptic strength can make a difference in your analysis. For EM data there are 3 common ways of reporting connection strength:

<ol>
    <li> <b> Binary connectivity:</b> whether two cells are connected at all, as shown above.</li>
    <li> <b> Synaptic count:</b> the total number of synapses that are part of the connection. This is typical of the Fly connectome where <code>count</code> captures much of the connection diversity. The modal <code>count</code> in mouse cortex is much lower.</li>
    <li> <b> Synaptic size:</b> the size of the postsynaptic density at every connection, generally aggregated as <code>sum</code> or <code>mean</code> for each unique connection.</li>
    
</ol>
How to take synapse size and number into account depends on the specific analysis. 

The `size` reported in the V1DD dataset measures the synaptic cleft as segmented by the automated classifier in voxels (3d pixels, a measure of volume). These are correlated to anatomical measures such as synaptic area and spine head volumes (for excitatory synapses). 

Let's replot the <b>square matrix</b> with the <code>sum</code> of synapses sizes between each connected pair.

</d>

In [ ]:
# get the ids of all proofread cells (same as for the binary connectivity)
proof_root_ids = axon_proof_root_ids[
    np.isin(axon_proof_root_ids, dendrite_proof_root_ids)
]

# Now with the aggregation function set to 'mean' synapse size
syn_mat = make_adjacency(syn_df, source_cell_index=proof_root_ids, aggfunc="sum")

# show the first 10 nonzero values in the array
row_indices, column_indices = np.nonzero(syn_mat)
print("Row indices:", row_indices[:10])
print("Column indices:", column_indices[:10])
print("Matrix values:", syn_mat[row_indices[:10], column_indices[:10]])

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

The matrix values are now **sum of size** rather than the True/False. It is useful to understand the full range of synaptic strength represented, plotted below first on a linear axis then a log axis

</d>

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 4), dpi=150)

# collect the synaptic weights that are non-zero
edge_weights = syn_mat[row_indices, column_indices]

# Histogram on linear-x
ax = axs[0]
sns.histplot(
    edge_weights,
    kde=True,
    bins=100,
    ax=ax,
)
ax.spines[["top", "right"]].set_visible(False)
ax.set(xlabel="Sum synapse size (voxels)", ylabel="Number of connections")

# Histogram on log-x
ax = axs[1]
sns.histplot(
    edge_weights,
    kde=True,
    bins=100,
    ax=ax,
    log_scale=True
)
ax.spines[["top", "right"]].set_visible(False)
ax.set(xlabel="Sum synapse size (voxels - log scale)", ylabel="Number of connections")

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

Replot the square matrix with `sum` of synapse size. 

</d>

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6), dpi=200)
sns.heatmap(
    syn_mat,
    cbar=True,
    cmap="Greys",
    xticklabels=False,
    yticklabels=False,
    square=True,
    vmax=3000,  # set the upper bound on color scale so that we can see more edges
    ax=ax,
    cbar_kws={"label": "Mean synapse size (nm)", "shrink": 0.5},
)
ax.spines[["left", "right", "top", "bottom"]].set_visible(True)
_ = ax.set(
    xlabel="Postsynaptic cell",
    ylabel="Presynaptic cell",
    title="Adjacency matrix between proofread cells",
)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

Depending on the connection type, number of synapses might be the more important measure (see [Dorkenwald et al., 2022](https://elifesciences.org/articles/76120) for deeper dive into synapse size and counts). 

</d>

In [ ]:
# generate adjacenty matrix with a count of synapses
syn_mat = make_adjacency(syn_df, proof_root_ids, aggfunc=lambda x: np.sum(x > 0))

# Histogram on linear-x
fig, ax = plt.subplots(figsize=(5, 4), dpi=150)
edge_weights = syn_mat[row_indices, column_indices]
sns.histplot(
    edge_weights,
    discrete=True,
    ax=ax,
)
ax.spines[["top", "right"]].set_visible(False)
ax.set(xlabel="Number of synapses per connection", ylabel="Number of connections")

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

Next we plot the <b>number of synapses per connection</b> or the <code>count</code>(we limit the colorbar to a maximum of 5 but the actual max number is much higher):

</div>

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6), dpi=200)
sns.heatmap(
    syn_mat,
    cbar=True,
    cmap="Greys",
    xticklabels=False,
    yticklabels=False,
    square=True,
    vmax=5,  # clip the color scale to 5 to see more edges
    ax=ax,
    cbar_kws={"label": "Number of synapses", "shrink": 0.5},
)
ax.spines[["left", "right", "top", "bottom"]].set_visible(True)
_ = ax.set(
    xlabel="Postsynaptic cell",
    ylabel="Presynaptic cell",
    title="Adjacency matrix between proofread cells",
)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

## Cell type tables

Identifying the putative ‘cell type’ from the EM morphology is a process that involves both manual and automatic classifications. Subsets of the dataset have been manually classified by anatomists at the Allen Institute, and these ground truth labels used to train and refine different automated ‘feature classifiers’ over time. 

Two of the most predictive features for broad cell type are:

<ul>
    <li>Soma location (in units of depth from pial surface). This captures <b>cortical layer diversity</b></li>
    <li>Soma size (in units of volume). This is a strong proxy of many types of <b>cell morpholigical diversity</b></li>
</ul>

<font size=4>Load the cell types data and plot all the cells by these two features</font>
    
</d>

In [ ]:
cell_df = pd.read_feather(f"{data_dir}/soma_and_cell_type_{mat_version}.feather")

cell_df.head()

In [ ]:
# Add a column that is soma depth in consistent units
cell_df['depth_um'] = cell_df['pt_position_trform_y'] / 1_000 # from nm to um

# Plot cell volume by depth
fig, ax = plt.subplots(figsize=(6, 6), dpi=150)
ax.tick_params(labelsize=14)
sns.scatterplot(
    data=cell_df,
    x="volume",
    y="depth_um",
    size=1,
    edgecolor=None,
    alpha=0.005,
    color="k",
    ax=ax,
    legend=False,
)
ax.invert_yaxis()
ax.set_xlabel(r"Volume ($\mu m^3$)", fontsize=16)
ax.set_ylabel(r"Depth ($\mu m$)", fontsize=16)
ax.set_xlim(0, 1000)
plt.show()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

### Using the automated cell types

Many of these automated cell type definitions were established and refined for the [MICrONS Dataset](https://www.microns-explorer.org/) including: 

<ul>
    <li> Perisomatic cell features <a href=https://www.nature.com/articles/s41586-024-07765-7>(Elabbady et al.)</a> </li>
    <li> Morphology and connectivity features <a href=https://www.nature.com/articles/s41586-024-07780-8>(Schneider-Mizell et al.)</a> </li>
</ul>

The process of applying these labels to the V1DD dataset is ongoing, but we will use cell typing from the <code>cell_type_multifeature_v1</code> table which labels cell types according to using soma, nucleus, dendrite, and spine features. 

Excitatory neurons are labeled by projection category (IT, ET, NP, CT, and SP for subplate). 

Inhibitory neurons follow labels based on targeting such as ITC (inhibitory targeting), STC (sparsely targeting), PTC (perisomatic targeting), and DTC (dendrite targeting), but might align more with molecular class than exact targeting for a given neuron, particularly among ITC/VIP cells.

*Note:* Cells here without a label will remain black. These are either non-neuronal cells or potential neurons with large segmentation errors that did not pass quality check
    
</d>

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6), dpi=150)
ax.tick_params(labelsize=14)
sns.scatterplot(
    data=cell_df,  # this is the dataframe
    x="volume",
    y="depth_um",
    size=1,
    edgecolor=None,
    alpha=0.005,
    color="k",
    ax=ax,
    legend=True,
)
sns.scatterplot(
    data=cell_df,
    x="volume",
    y="depth_um",
    size=1,
    edgecolor=None,
    alpha=0.1,
    color="k",
    ax=ax,
    legend=True,
    hue="cell_type",
)
ax.invert_yaxis()
ax.set_xlabel(r"Volume ($\mu m^3$)", fontsize=16)
ax.set_ylabel(r"Depth ($\mu m$)", fontsize=16)
ax.set_xlim(0, 1000)
plt.show()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

## Sorting the synapse matrix with cell types
    
Let's combine the synaptic connecitivity with the cell type information. Below we provide logic for sorting a connectivity matrix using a list of labels.    
    
</d>

In [ ]:
# Filter the proofread ids for those with a known cell type
proof_root_ids = np.intersect1d(proof_root_ids, cell_df["pt_root_id"].values)

# Filter the cell type table for those with a proofread root id
proof_cell_df = cell_df.set_index("pt_root_id").loc[proof_root_ids]
proof_cell_df = proof_cell_df.query("cell_type.notna()")

# NOTE: the adjacency matrix will be sorted according to this dataframe's index, so
# we'll sort it by soma depth
proof_cell_df = proof_cell_df.sort_values("pt_position_trform_y")

# Make adjacency, now including soma depth and cell type
syn_mat = make_adjacency(syn_df, proof_cell_df, aggfunc="sum")
syn_mat

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

NOTE: there are slightly fewer cells in this adjacency matrix, because we are filtering for cells that have complete information in the cell table above.

</div>

In [ ]:
sns.set_context("paper", font_scale=1.5)
fig, ax = plt.subplots(figsize=(6, 6), dpi=200)
sns.heatmap(
    syn_mat,
    cbar=True,
    cmap="Greys",
    xticklabels=False,
    yticklabels=False,
    square=True,
    vmax=3000,
    ax=ax,
    cbar_kws={"label": r"Sum synapse size (voxels)", "shrink": 0.5},
)
ax.spines[["left", "right", "top", "bottom"]].set_visible(True)
_ = ax.set(
    xlabel="Postsynaptic cell (by depth)",
    ylabel="Presynaptic cell (by depth)",
    title="Adjacency matrix between proofread cells",
)

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Task 4:</b> There is a large amount of structure in the connectivity, just based on soma depth! However, some rows are very narrow in their connectivty while others are very wide.
    
<p><b>Discussion:</b> What are some explanations for why the connectivity plot has a strong diagonal component? In contrast, what does the wide connectivity in a row/column mean </p>
    
</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

### Add cell type logic to adjacency plot

What about the effect of cell types on the connectivity matrix? We can add colors and subplots by cell type to make this matrix more tractable. We have packaged away the logic for this in `adjacencyplot` which can group and sort the adjacency matrix by metadata in the cell table.

The logic here is similar to the plotting package `seaborn`, where different labels within the data can be leveraged to control different elements.
    
</d>

In [ ]:
from utils import adjacencyplot, cell_type_palette

sns.set_context('paper', font_scale=1.0)

fig, ax = plt.subplots(figsize=(8, 8), dpi=200)

# render the adjacency plot
adjacencyplot(
    syn_mat,  # data values for the points
    nodes=proof_cell_df,  # data to organize the x and y axis
    groupby=["cell_type_coarse", "cell_type"],  # categorical variables to organize by
    sortby="pt_position_y",  # sort within groups by variable
    node_palette=cell_type_palette,
    title="Proofread connectivity",
    edge_palette="Greys",
    hue_norm=(0, 2000),  # normalize the color scale for the edges
    ax=ax,
    label_fontsize="xx-small",
    title_fontsize="medium",
    arc_labels=False
)

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Task 5:</b> Now we have restructured the connectivity based on cell type. Find one of your example cells from earlier and identify it's row or column. Hint: query the <code>proof_cell_df</code> by your <code>root_id</code>

<b>Discussion:</b> Describe the connectivity for your cell by who it targets and who it recieves inputs from.

Then, for each row describe that cell type by it's major targets. Do you think there are more subgroups within the cell types?
    
</div>

In [ ]:
# Query the cell type of your root_id:


<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

## Connection specificity onto compartment types
Up until now we have considered all synapses between any two cells, but different synapses target different compartments on the post synaptic cell. 

![](../../figures/sss-diagram.png)

For example, it is common understanding that:

<ol>
    <li> Synapses between excitatory->excitatory cells are onto spines</li>
    <li> Synapses between inhibitory->excitatory cells are onto dendritic shaft</li>
    <li> Synapses onto inhibitory cells are onto dendritic shaft</li>
    <li> Basket cells target soma synapses </li>
</ol>


Let's test this textbook understanding in the real data. 

<font size=4> For this you will need to load in data about the target compartment of the synapse, one of: spine, shaft, soma </font>
    
</d>

In [ ]:
# Now load the table that includes the target structure and combine to your synapse matrix
target_structure = pd.read_feather(
    pjoin(data_dir, f"syn_label_df_all_to_proofread_to_all_{mat_version}.feather")
)["tag"]
target_structure

# Select the synapses between proofread cells
proof_proof_syn_table = filter_synapse_table(
    syn_df, pre_root_ids=proof_root_ids, post_root_ids=proof_root_ids
).copy().set_index('id')

# Combine the target information to the proofread synapses table
proof_proof_syn_table["target_structure"] = target_structure
proof_proof_syn_table["target_structure"] = proof_proof_syn_table[
    "target_structure"
].fillna("unknown")

print(proof_proof_syn_table["target_structure"].value_counts())
proof_proof_syn_table.head()

In [ ]:
## Plot just synapses onto spine

fig, ax = plt.subplots(figsize=(8, 8), dpi=200)

category = 'spine' # This is synapses onto spines

category_syns_df = proof_proof_syn_table.query(f"target_structure == '{category}'")
category_syn_mat = make_adjacency(
    category_syns_df, source_cell_index=proof_cell_df, aggfunc="sum"
)
# render the adjacency plot
adjacencyplot(
    category_syn_mat,  # data values for the points
    nodes=proof_cell_df,  # data to organize the x and y axis
    groupby=["cell_type_coarse", "cell_type"],  # categorical variables to organize by
    sortby="pt_position_y",  # sort within groups by variable
    node_palette=cell_type_palette,
    title="Proofread connectivity",
    edge_palette="Greys",
    hue_norm=(0, 2000),  # normalize the color scale for the edges
    ax=ax,
    label_fontsize="xx-small",
    title_fontsize="medium",
    arc_labels=False
)


<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Task 6:</b> Reproduce the plot above from 'spine' for 'shaft' and 'soma'

<b>Discussion:</b> For each of the common cell-type connectivity assumptions above, which hold up in the data? 

Find one thing that does not match your expectation, and write it on the whiteboard.
</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

## Bonus at the bottom of the workshop

Treasure! For scrolling so far.

If you are interested in exploring further, here is some code for computing the number and proportion of edges that go between cell types for each different target structure. This is useful for summarizing the observations above. Note that this is just chaining together some Pandas operations like groupby that we already saw.
    
</d>

In [ ]:
proof_proof_syn_table['pre_cell_type'] = proof_proof_syn_table['pre_pt_root_id'].map(
    proof_cell_df["cell_type"]
)
proof_proof_syn_table['post_cell_type'] = proof_proof_syn_table['post_pt_root_id'].map(
    proof_cell_df["cell_type"]
)
category_edge_info = (
    proof_proof_syn_table.groupby(["pre_cell_type", "post_cell_type", "target_structure"])
    .size()
    .rename("edge_count")
    .to_frame()
    .reset_index()
)
category_edge_info["edge_prop"] = category_edge_info["edge_count"] / (
    category_edge_info.groupby(["pre_cell_type", "post_cell_type"])[
        "edge_count"
    ].transform("sum")
)
category_edge_info